# Counter Strike Global Offensive Match Result Prediction
- **Leonardo Valerio Morales 771030**
- **Luis Felipe Dobner Henriques 771036**

This notebook executes data pre-processing and predictive analysis of Counter Strike Global Offensive Matches.

# Data Loading
This step will create the tables and schemas in both databases and load the concerning data from the dataset into Neo4j and Apache Kudu.

In [ ]:
# TODO - Load Data

# Data Pre Processing
This step will use the loaded data in Apache Kudu to pre process specific team win rates, and insert that data into de Neo4j database for later use during result prediction.

In [ ]:
# TODO - Pre process data in apache kudu and send to Neo4j

# Result Prediction
This is the main step to be used in result prediction, it will load data from Neo4j, while simultaneously processing raw generic data present in Apache Kudu. Results from both databases will then be fed into an algorithm that predicts the winner of the match.

In [ ]:
# TODO - Define functions that will interpret the match input and call multiple queries to fetch the win rates
# TODO - Define the algorithm that takes the win rates and outputs final prediction
# TODO - Define an interface for the user to call the interpreter function

# Bonus Queries
This section is used to call other queries not linked to result prediction like objective queries or player statistics.

In [ ]:
# low priority TODO 
# Define functions to execute objective queries in Neo4j
# Define functions to execute generic statistics queries in Apache Kudu
# Define interface to call above functions